In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms

# Define the train_model function if not already defined
def train_model(output_path, model, dataloaders, criterion, optimizer, device, num_epochs=20):
    best_acc = 0.0
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        corrects = 0

        for inputs, labels in dataloaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = corrects.double() / dataset_sizes['train']

        print('Epoch {} Train Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, epoch_loss, epoch_acc))

        # Validation phase
        model.eval()
        running_loss = 0.0
        corrects = 0

        for inputs, labels in dataloaders['val']:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['val']
        epoch_acc = corrects.double() / dataset_sizes['val']

        print('Epoch {} Val Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, epoch_loss, epoch_acc))

        # Save the model if it has the best validation accuracy
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            torch.save(model.state_dict(), output_path + '_best.pth')

    print('Best Val Acc: {:4f}'.format(best_acc))

# Define transforms
data_transforms = {
    "train": transforms.Compose([transforms.RandomResizedCrop(224),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    "val": transforms.Compose([transforms.Resize(256),
                               transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    "test": transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
}

# Set data directory
data_dir = "tiny-imagenet-200"

# Set number of workers
num_workers = {"train": 2, "val": 0, "test": 0}

# Load datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ["train", "val", "test"]}

# Create data loaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64, shuffle=True, num_workers=num_workers[x]) for x in ["train", "val", "test"]}

# Dataset sizes
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "val", "test"]}

# Load ResNet-18
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = models.resnet18(pretrained=True).to(device)

# Freeze parameters
for param in model_ft.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit the number of classes in your dataset
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(image_datasets['train'].classes)).to(device)

# Loss Function
criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized
optimizer_ft = optim.SGD(model_ft.fc.parameters(), lr=0.001, momentum=0.9)

# Train
best_epoch = train_model(
    output_path="ResNet_pretrained",
    model=model_ft,
    dataloaders=dataloaders,
    criterion=criterion,
    optimizer=optimizer_ft,
    device=device,
    num_epochs=20,
)


Epoch 1 Train Loss: 4.6799 Acc: 0.1130
Epoch 1 Val Loss: 6.2128 Acc: 0.0061
Epoch 2 Train Loss: 3.8498 Acc: 0.2439
Epoch 2 Val Loss: 6.9712 Acc: 0.0059
Epoch 3 Train Loss: 3.4807 Acc: 0.2875
Epoch 3 Val Loss: 7.4260 Acc: 0.0057
Epoch 4 Train Loss: 3.2883 Acc: 0.3075
Epoch 4 Val Loss: 7.8106 Acc: 0.0054
Epoch 5 Train Loss: 3.1742 Acc: 0.3224
Epoch 5 Val Loss: 7.9218 Acc: 0.0057
Epoch 6 Train Loss: 3.0995 Acc: 0.3294
Epoch 6 Val Loss: 7.9827 Acc: 0.0060
Epoch 7 Train Loss: 3.0412 Acc: 0.3383
Epoch 7 Val Loss: 8.3475 Acc: 0.0055
Epoch 8 Train Loss: 3.0028 Acc: 0.3408
Epoch 8 Val Loss: 8.6586 Acc: 0.0048
Epoch 9 Train Loss: 2.9652 Acc: 0.3484
Epoch 9 Val Loss: 8.6671 Acc: 0.0055
Epoch 10 Train Loss: 2.9464 Acc: 0.3492
Epoch 10 Val Loss: 8.5941 Acc: 0.0058
Epoch 11 Train Loss: 2.9276 Acc: 0.3512
Epoch 11 Val Loss: 8.8192 Acc: 0.0056
Epoch 12 Train Loss: 2.9038 Acc: 0.3561
Epoch 12 Val Loss: 8.6012 Acc: 0.0057
Epoch 13 Train Loss: 2.8932 Acc: 0.3577
Epoch 13 Val Loss: 8.6597 Acc: 0.0061
Epoc